<a href="https://colab.research.google.com/github/RahafAlharthi/Projects/blob/master/YOLO_Practice_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Highway Vehicle Counting Practice Exercise


In this exercise, you will use the YOLO (You Only Look Once) object detection model to analyze a video of highway traffic.
Your task is to count the number of cars that are leaving the highway (coming toward the camera) on the right side and the number of cars that are joining the road on the left side. The video can be found under `Datasets/Example.mp4`.

## Objectives
- Load and process a video using OpenCV.
- Use the YOLO model to detect vehicles in each frame.
- Use OpenCV to manulate the video.
- Track vehicles as they move through the video frames.
- Count the number of vehicles leaving the highway on the right side.
- Count the number of vehicles joining the road on the left side.

## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [40]:
!pip install ultralytics

In [41]:
!pip install gdown

In [42]:
from ultralytics import YOLO
import os
from IPython.display import Image, display
from IPython import display
display.clear_output()

## Load the YOLO Model

You will first need to load the YOLO model. You can use a pre-trained YOLO model for this task.
Write the code to load the YOLO model below:


In [43]:
import cv2
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [44]:
video_path = '/content/Example.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [45]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [46]:
print(width)
print(height)
print(fps)

1280
720
25


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes and the counted cars on both sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [47]:
output_path = '/content/output_video_with_yolov8.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Process Video Frames and Identify Vehicles on the Right and Left Sides

For each frame in the video, use the YOLO model to detect and track vehicles. You'll need to write a loop that processes each frame and applies the YOLO model.
In each frame, after detecting the vehicles, determine whether they are on the left or right side of the highway.
You can use the position of the bounding boxes provided by YOLO to do this.
* The video should display bounding boxes around the detected objects.
* The video should display the confidence along side with the object id and class id of each detected and tracked object.
* The video display the number of vehicles on the left side.
* The video display the number of vehicles on the right side.
* The video should display the line in which you counted the objects that have crossed it and counted.

In [48]:
color = (0, 255, 0)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_thickness = 1

unique_ids = set()
roi_left = (0, 0, width // 2, height)
roi_right = (width // 2, 0, width, height)
line_x = width // 2
unique_ids_left = set()
unique_ids_right = set()
ret = True
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    results = model.track(frame, persist=True)

    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0
            obj_id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1

            unique_ids.add(obj_id)

            x1, y1, x2, y2 = map(int, bbox)
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            label = f'ID: {obj_id} Class: {class_id} Conf: {conf:.2f}'

            cv2.putText(frame, label, (x1, y1 - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)

            # check if the vehicle is in the left or right ROI
            if roi_left[0] <= center_x <= roi_left[2] and roi_left[1] <= center_y <= roi_left[3]:
                unique_ids_left.add(obj_id)
            elif roi_right[0] <= center_x <= roi_right[2] and roi_right[1] <= center_y <= roi_right[3]:
                unique_ids_right.add(obj_id)

    cv2.line(frame, (line_x, 0), (line_x, height), (255, 0, 0), 2)# line between right and left
    unique_count_label = f'Total Unique Objects: {len(unique_ids)}'# total objects
    cv2.putText(frame, unique_count_label, (10, height - 20), font, font_scale, color, font_thickness, cv2.LINE_AA)

    left_cars =  f'Left Cars: {len(unique_ids_left)}'# print counter of left cars
    cv2.putText(frame, left_cars, (10, height - 40), font, font_scale, color, font_thickness, cv2.LINE_AA)

    right_cars =  f'Right Cars: {len(unique_ids_right)}'# print counter of right cars
    cv2.putText(frame, right_cars, (10, height - 60), font, font_scale, color, font_thickness, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()


0: 384x640 5 cars, 9.2ms
Speed: 1.8ms preprocess, 9.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 8.3ms
Speed: 2.3ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 7.0ms
Speed: 1.6ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 6.8ms
Speed: 2.0ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 6.7ms
Speed: 1.7ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 6.7ms
Speed: 2.0ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 6.9ms
Speed: 2.1ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 8.9ms
Speed: 2.1ms preprocess, 8.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 c

## Save and Submit Your Work

In the actual exam you will be asked to submit both the notebook and the output video